In [1]:
import requests
import pandas as pd
import plotly.express as px
import json
import sqlalchemy

In [2]:
census_api_key = ""

In [3]:
pd.options.display.max_rows = None

In [4]:
cities = ['New York city, New York', 'Los Angeles city, California', 'San Francisco city, California', 'Miami city, Florida', 'Seattle city, Washington', 'Atlanta city, Georgia', 'Denver city, Colorado', 'Chicago city, Illinois', 'Dallas city, Texas']

In [5]:
def json_to_dataframe(response):
    """
    Convert response to dataframe
    """
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [6]:
url = "https://api.census.gov/data/2019/pep/population?get=NAME,POP&for=place:*&key={0}".format(census_api_key)
response = requests.request("GET", url)

In [7]:
response

<Response [200]>

In [8]:
all_cities = json_to_dataframe(response).sort_values("NAME")
all_cities["POP"] = all_cities["POP"].astype(float)
all_cities.head()

,NAME,POP,state,place
0,"Abbeville city, Alabama",2560.0,01,00124
2454,"Abbeville city, Georgia",2684.0,13,00184
7052,"Abbeville city, Louisiana",12038.0,22,00100
15619,"Abbeville city, South Carolina",5014.0,45,00100
8978,"Abbeville town, Mississippi",423.0,28,00100


In [9]:
filtered_cities = all_cities.loc[all_cities['NAME'].isin(cities)].reset_index()
cities_population = filtered_cities[['NAME', 'POP']]


In [10]:
cities_population = cities_population.rename(columns = {'NAME':'location', 'POP':'population'}).reset_index(drop=True)
cities_population.head(20)

,location,population
0,"Atlanta city, Georgia",506811.0
1,"Chicago city, Illinois",2693976.0
2,"Dallas city, Texas",1343573.0
3,"Denver city, Colorado",727211.0
4,"Los Angeles city, California",3979576.0
5,"Miami city, Florida",467963.0
6,"New York city, New York",8336817.0
7,"San Francisco city, California",881549.0
8,"Seattle city, Washington",753675.0


In [11]:
cities_population['location'] = cities_population['location'].replace({'New York city, New York':'New York City', 'Los Angeles city, California':'Los Angeles', 'San Francisco city, California':'San Francisco', 'Miami city, Florida':'Miami',
       'Seattle city, Washington':'Seattle', 'Atlanta city, Georgia':'Atlanta', 'Denver city, Colorado':'Denver', 'Chicago city, Illinois':'Chicago', 'Dallas city, Texas':'Dallas'})
cities_population
                                                          

,location,population
0,Atlanta,506811.0
1,Chicago,2693976.0
2,Dallas,1343573.0
3,Denver,727211.0
4,Los Angeles,3979576.0
5,Miami,467963.0
6,New York City,8336817.0
7,San Francisco,881549.0
8,Seattle,753675.0


In [12]:
cities_list = [
    ['New York City', 40.7128, -74.0060],
    ['Los Angeles' , 34.0522, -118.2437],
    ['San Francisco', 37.7749, -122.4194],
    ['Miami', 25.7617, -80.1918],
    ['Seattle', 47.6062, -122.3321],
    ['Atlanta', 33.7490, -84.3880],
    ['Denver', 39.7392, -104.9903],
    ['Chicago', 41.8781, -87.6298],
    ['Dallas', 32.7767, -96.7970]
]

In [13]:
city_coordinates = {'location':[], 'latitude':[], 'longitude':[]}

In [14]:
for city in cities_list:
    city_name = city[0]
    latitude = city[1]
    longitude = city[2]
    city_coordinates['location'].append(city[0])
    city_coordinates['latitude'].append(city[1])
    city_coordinates['longitude'].append(city[2])

In [15]:
cities_cordinates = pd.DataFrame(city_coordinates)

In [16]:
database_connection_string = 'sqlite:///'

In [17]:
engine = sqlalchemy.create_engine(database_connection_string)
engine

Engine(sqlite:///)

In [18]:
cities_population.to_sql('cities_population', engine, index=False, if_exists='replace')

9

In [19]:
restaurant_data = pd.read_csv("RestaurantData.csv")

In [20]:
restaurant_data.to_sql('restaurant_data', engine, index=False, if_exists='replace')

144

In [21]:
cities_cordinates.to_sql('cities_cordinates', engine, index=False, if_exists='replace')

9

In [22]:
engine.table_names()


C:\Users\Matheus\AppData\Local\Temp\ipykernel_32772\509743474.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['cities_cordinates', 'cities_population', 'restaurant_data']

In [23]:
query = """
SELECT rd.location, rd.cuisine, rd.price, rd.count, cp.population, cc.latitude, cc.longitude
FROM restaurant_data AS rd
LEFT JOIN cities_population AS cp
ON rd.location = cp.location
LEFT JOIN cities_cordinates AS cc
ON rd.location = cc.location;
"""

In [24]:
joined_df = pd.read_sql_query(query, con=engine)

In [25]:
joined_df.head(40)


,location,cuisine,price,count,population,latitude,longitude
0,New York City,Indian,1,304,8336817.0,40.7128,-74.0060
1,New York City,Indian,2,793,8336817.0,40.7128,-74.0060
2,New York City,Indian,3,31,8336817.0,40.7128,-74.0060
3,New York City,Indian,4,4,8336817.0,40.7128,-74.0060
4,New York City,Mexican,1,1700,8336817.0,40.7128,-74.0060
5,New York City,Mexican,2,3400,8336817.0,40.7128,-74.0060
6,New York City,Mexican,3,319,8336817.0,40.7128,-74.0060
7,New York City,Mexican,4,33,8336817.0,40.7128,-74.0060
8,New York City,Italian,1,2500,8336817.0,40.7128,-74.0060
9,New York City,Italian,2,5200,8336817.0,40.7128,-74.0060


In [26]:
new_join_df = joined_df[['location', 'cuisine', 'price', 'count']]


In [27]:
pivot_df = joined_df.pivot_table(index=['location', 'cuisine', 'population', 'latitude', 'longitude'], columns='price', values='count')
pivot_df.columns = ['price1', 'price2', 'price3', 'price4']

In [28]:
pivot_df.reset_index(inplace=True)

In [29]:
pivot_df

,location,cuisine,population,latitude,longitude,price1,price2,price3,price4
0,Atlanta,Chinese,506811.0,33.7490,-84.3880,324,397,23,3
1,Atlanta,Indian,506811.0,33.7490,-84.3880,50,131,2,0
2,Atlanta,Italian,506811.0,33.7490,-84.3880,376,706,77,13
3,Atlanta,Mexican,506811.0,33.7490,-84.3880,514,793,39,5
4,Chicago,Chinese,2693976.0,41.8781,-87.6298,530,600,48,7
5,Chicago,Indian,2693976.0,41.8781,-87.6298,81,202,6,2
6,Chicago,Italian,2693976.0,41.8781,-87.6298,1100,2400,216,26
7,Chicago,Mexican,2693976.0,41.8781,-87.6298,1500,1900,114,18
8,Dallas,Chinese,1343573.0,32.7767,-96.7970,305,377,20,4
9,Dallas,Indian,1343573.0,32.7767,-96.7970,79,141,0,0


In [123]:
pivot_df.to_csv('restaurants_pivot.csv')